# 第六組 第四題 KNN 補值實作

葉高緯 製作
首先`import`需要用到的東西

In [1]:
import pandas as pd
import warnings
from sklearn import preprocessing
from sklearn.metrics.pairwise import nan_euclidean_distances
from sklearn.impute import KNNImputer
from sklearn.metrics import mean_squared_error
from collections import Counter as ct
import random
import numpy as np

warnings.filterwarnings('ignore')

讀取檔案

In [2]:
le = preprocessing.LabelEncoder()
data = pd.read_csv("Titanic_train.csv")


先去除我們不要的欄位，
乘客編號、姓名、票號在KNN是不要的，
艙位的資料遺漏太多也去掉。

同時也把性別轉換一下。


In [3]:
data = data.drop(columns=['PassengerId', 'Name', 'Ticket', 'Cabin'])
data['Sex'] = le.fit_transform(data['Sex'])


先處理 Embarked，
先轉換資料成連續型，

因為只缺失兩筆，
而且這資料感覺也跟 Age 不太有關，
所以直接以中位數填補，
相當於 KNN 的 K = 樣本數。


In [4]:
# 以中位數填補缺失數值
data.loc[data["Embarked"] == "S", "Embarked"] = 0
data.loc[data["Embarked"] == "C", "Embarked"] = 1
data.loc[data["Embarked"] == "Q", "Embarked"] = 2
data['Embarked'] = data['Embarked'].fillna(data["Embarked"].median())


現在把準備工作都做好了，
那該怎麼找到想要的 K？
如果我想用 MSE 需要有正確的數值。

那正確的數值哪裡來？
只剩下原本那八成的 Age。

所以我想是這樣，
把已存在的數值用 MSE 找出最佳的 K，
用這個 K 帶入大概就是最好的結果，
所以我們繼續處理資料。


In [5]:
# 去除遺失的資料，重製 index

data = data.dropna()
data.index = range(len(data))
data

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,1,22.0,1,0,7.2500,0.0
1,1,1,0,38.0,1,0,71.2833,1.0
2,1,3,0,26.0,0,0,7.9250,0.0
3,1,1,0,35.0,1,0,53.1000,0.0
4,0,3,1,35.0,0,0,8.0500,0.0
...,...,...,...,...,...,...,...,...
709,0,3,0,39.0,0,5,29.1250,2.0
710,0,2,1,27.0,0,0,13.0000,0.0
711,1,1,0,19.0,0,0,30.0000,0.0
712,1,1,1,26.0,0,0,30.0000,1.0


接下來我們開始準備動手處理 MSE，
隨機遺失的資料數量選擇為140，
因為大概等於總體的兩成。

In [6]:
def MyMSE(k):
    a = random.sample(list(data.index), 140)
    missing_data = data.copy()
    missing_data['Age'].iloc[a] = np.nan

    imputer = KNNImputer(n_neighbors=k, weights='distance', add_indicator=True)
    pred_data = pd.DataFrame(imputer.fit_transform(missing_data))

    y_true1 = data['Age']
    y_pred1 = pred_data[3]
    MSE_h = mean_squared_error(y_true1, y_pred1)
    return MSE_h


接下來打算把所有考慮的k都跑一次，
看看哪一個k能跑出最小的MSE。

k值的考慮範圍很重要，
範圍大的k就不精準，
而且跑出來的最小k差距很大，
30~250都有可能，
我們要找到一個範圍剛好讓差距很明顯。

我想1~5非常合適，
有明顯的差距，
我猜想在鐵達尼號旅遊的大概都是家族旅行，
一個家庭的組成年紀應該差不多，
所以那個家族的老人也會跟其他家族的老人差不多，
小孩也是，
家庭的人數也在5個人左右。

In [7]:
def find_min_MSE():
    res = []
    for k in range(1,6):
        res.append(MyMSE(k))
    res_min = np.min(res)
    
    for i in range(len(res)):
        if res_min == res[i]:
            return i+1

重複一百次這個隨機行為，
統計看看哪個k的MSE最小，
發現k = 5都會最突出。

In [8]:
rec = []
for i in range(100):
    rec.append(find_min_MSE())
count = ct(rec)
print(dict(count))

{4: 27, 5: 45, 2: 9, 3: 19}


最後回到本題，
k = 5的話應該是最好的。